In [ ]:
import sys
sys.path.append('/usr/local/lib/python3.6/site-packages/')
sys.path.append('../')

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from models.Noisy_models.Noisy_abstract_model import Noisy_abstract_model, Null_model
from utils.sequence_utils import *
import editdistance
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


In [ ]:
import RNA
RAA="UGCA"

In [ ]:
target='GAACGAGGCACAUUCCGGCUCGCCCGGCCCAUGUGAGCAUGGGCCGGACCCCGUCCGCGCGGGGCCCCCGCGCGGACGGGGGCGAGCCGGAAUGUGCCUC'
wt="AGUC"*20

In [ ]:
from models.Ground_truth_oracles.RNA_landscape_models import RNA_landscape_binding

In [ ]:
len(wt)

In [ ]:
wt=generate_random_sequences(100,1,alphabet=RAA)[0]

In [ ]:
wt

In [ ]:
base_landscape=RNA_landscape_binding(target)
#base_landscape=Conserved_landscape(base_landscape, wt[:2],0)
base_landscape.get_fitness(wt)

In [ ]:
noisy_landscape= Noisy_abstract_model(base_landscape,signal_strength=0.5)
noisy_landscape = Null_model(base_landscape)
noisy_landscape.reset([wt])

In [ ]:
noisy_landscape.get_fitness(wt)

In [ ]:
len("AUGUUUCUUUUAUUUAUCUGAGCAUGGGCGGGGCAUUUGCCCAUGCAAUU")

In [ ]:
noisy_landscape.get_fitness("ACCGUGGUUCCAGGUGAAAACGAUAUGCUACCUGCUAAACGAGAAAGACGGGAACUUGAUAAGCGAGCAAAGCCAGUAUUUAGUUACAGCAGGAAUGUCC")

In [ ]:
noisy_landscape.cost

In [ ]:
def walk_out(start,steps):
    seqs=[start]
    for i in range(steps):
        seqs.append(random.choice(generate_single_mutants(seqs[-1],alphabet=RAA)))

    return seqs

In [ ]:
walk=walk_out(wt,15)

In [ ]:
fitnesses=[base_landscape.get_fitness(x) for x in walk]

In [ ]:
noisy_fitnesses=[]
indices=[]
ndf=pd.DataFrame()
for i in range(1000):
    noisy_landscape.reset([wt])
    plt.scatter(range(16), [noisy_landscape.get_fitness(x) for x in walk],c="r",alpha=0.25,s=2)
    noisy_fitnesses.extend([noisy_landscape.get_fitness(x) for x in walk])
    indices.extend([x for x in range(16)])
    
len(indices),len(noisy_fitnesses)
ndf["distance"]=indices
ndf["nfit"]=noisy_fitnesses
plt.plot(fitnesses)

#plt.ylim(0,1)

In [ ]:
plt.plot(fitnesses)

sns.lineplot(x="distance",y="nfit",data=ndf,color="r")
#plt.ylim(0.4,0.8)

In [ ]:
some_sequences= [generate_random_mutant(wt,0.05, RAA) for i in range(0,1000)]
plt.scatter([base_landscape.get_fitness(s) for s in some_sequences],\
           [noisy_landscape.get_fitness(s) for s in some_sequences])

In [ ]:
from explorers.base_explorer import Base_explorer, Random_explorer

In [ ]:
noisy_landscape.reset([wt])

In [ ]:
bexp= Random_explorer(alphabet="UCGA",mu=0.03)
bexp.set_model(noisy_landscape)

In [ ]:
#bexp.batches

In [ ]:
bexp.run(5,overwrite=True)

In [ ]:
data=pd.read_csv("./simulations/Random_mu0.03.csv",index_col=False)

In [ ]:
data.head()

In [ ]:


plt.scatter(data["true_score"],data["model_score"])

In [ ]:
from scipy.stats import pearsonr
for i in data.batch.unique():
    sub_data=data[data.batch==i] #must also group by model type and landscape id
    print(len(sub_data))
    print (pearsonr(sub_data["true_score"],sub_data["model_score"]))

In [ ]:
def get_top_sequence_per_batch(data):
    top_per_batch=[]
    for i in data.batch.unique():
        sub_data=data[data.batch==i] #must also group by model type and landscape id
        #print (sub_data.true_score.max())
        top_per_batch.append(sub_data.true_score.max())
    return (top_per_batch)
    

In [ ]:
get_top_sequence_per_batch(data)

In [ ]:
from explorers.evolutionary_explorers import WF

In [ ]:
wt= "AUGUUUCUUUUAUUUAUCUGAGCAUGGGCGGGGCAUUUGCCCAUGCAAUU"

In [ ]:
noisy_landscape.reset([wt])

In [ ]:
bexp= WF(mu=0.01, recomb_rate=0.1,beta=100)
bexp.set_model(noisy_landscape)

In [ ]:
bexp.run(15,overwrite=True,verbose=True)

In [ ]:
data=pd.read_csv("./simulations/WF_mu0.01_r0.1_rho1_beta100.csv",index_col=False)

In [ ]:
data.tail()

In [ ]:
get_top_sequence_per_batch(data)

In [ ]:
from scipy.stats import pearsonr
for i in data.batch.unique():
    sub_data=data[data.batch==i] #must also group by model type and landscape id
    print(len(sub_data))
    print (pearsonr(sub_data["true_score"],sub_data["model_score"]))

In [ ]:
from explorers.elitist_explorers import XE_IS, Greedy

In [ ]:
noisy_landscape.reset([wt])

In [ ]:
bexp= XE_IS(beta=100, recomb_rate=0.1, threshold=0.05)
bexp.set_model(noisy_landscape)

In [ ]:
bexp.run(10,overwrite=True,verbose=True)

In [ ]:
noisy_landscape.reset([wt])
bexp= Greedy(recomb_rate=0.1, threshold=0.05, mu=0.02, virtual_screen=100)
bexp.set_model(noisy_landscape)

In [ ]:
bexp.run(11,overwrite=True,verbose=True)

In [ ]:
from utils.model_architectues import Linear, NLNN, CNNa

In [ ]:
cnn_model=NLNN(len(wt), alphabet="UCGA")

In [ ]:
cnn_model.seq_len

In [ ]:
from models.Noisy_models.Neural_network_models import NN_model

In [ ]:
nnModel=NN_model(base_landscape,cnn_model)

In [ ]:
nnModel.reset()

In [ ]:
some_sequences= [generate_random_mutant(wt,0.05, RAA) for i in range(0,1000)]
some_other_sequences= [generate_random_mutant(wt,0.02, RAA) for i in range(0,1000)]


In [ ]:
nnModel.measure_true_landscape(some_sequences)

In [ ]:
nnModel.update_model(some_sequences)

In [ ]:
plt.scatter([base_landscape.get_fitness(s) for s in some_other_sequences],\
           [nnModel.get_fitness(s) for s in some_other_sequences])

In [ ]:
pearsonr([base_landscape.get_fitness(s) for s in some_other_sequences],\
           [nnModel.get_fitness(s) for s in some_other_sequences])

In [ ]:
from models.Ground_truth_oracles.RNA_landscape_models import *

In [ ]:
rna_l_constrctr.import_landscapes("data/RNA_landscapes/RNA_landscape_config.yaml")

In [ ]:
new_landscape = rna_l_constrctr.construct(rna_l_constrctr.imported_landscapes[0])

In [ ]:
new_landscape.get_fitness('AUGGGCCGGACCCC')

In [ ]:
rna_l_constrctr = RNA_landscape_constructor() 
rna_l_constrctr.load_landscapes("data/RNA_landscapes/RNA_landscape_config.yaml")
#new_landscape ,starts,lid=  rna_l_constrctr.construct(rna_l_constrctr.loaded_landscapes[-1])

In [ ]:
for landscape in rna_l_constrctr.imported_landscapes:
    #print (landscape)
    new_landscape,starts,lid=  rna_l_constrctr.construct(landscape)
    print (lid)
    for start in starts: 
        print (new_landscape.get_fitness(starts[start]), start)

In [ ]:
from models.Ground_truth_oracles.TF_binding_landscape_models import *

In [ ]:
TFc=TF_binding_landscape_constructor()

In [ ]:
TFc.load_landscapes()

In [ ]:
TFgen= TFc.generate_from_loaded_landscapes()

In [ ]:
for landscape in TFgen:
    print (landscape["landscape_id"])
    for seq in landscape["starting_seqs"]:
        print (landscape["landscape_obj"].get_fitness(landscape["starting_seqs"][seq]))

In [ ]:
#data=pd.read_csv("./simulations/eval/CRI/Greedy_mu0.02_tr0.05_r0.1_rho1.csv",index_col=False)
data=pd.read_csv("./simulations/eval/EF/Greedy_mu0.02_tr0.05_r0.1_rho1.csv",index_col=False)


In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
data.groupby( 'mo_ratio').max()

In [ ]:
plt.figure(figsize=(14,5))
sns.boxplot(x='mo_ratio',y="true_score",data=data)

In [ ]:
sns.lineplot(x="measurement_cost",y="true_score",hue="model_type",data=data)

In [ ]:
sns.lineplot(x="measurement_cost",y="true_score",hue="model_type",
             data=data.groupby(["measurement_cost","landscape_id","model_type"]).max())

In [ ]:
data.groupby(["measurement_cost","landscape_id","model_type"]).max()

In [ ]:
import timeit